# Make animation of an illuminated planet on an eccentric orbit

M Kenworthy // 2023 June 09

This makes a set of images that can be combined into an animation showing a planet orbiting a star, with the disk of the planet showing the correct illumination.

The planet does not block the orbit path correctly, this would require additional calculation of the intercept of the orbit with the disk of the planet.

In [ ]:
import numpy as np
import astropy.units as u
import astropy.constants as c
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from phases import *

Next cell is all the subroutines for orbital calculation

In [ ]:
#%matplotlib agg
%matplotlib inline
# Inner working angle for the imaging
iwa = 0.1 # arcsec

# set up our orbit
# CAUTION! you need to use Kepler's laws and know M1 and M2 to set P and a consistently :)
#
P = 1.1 * u.year
tperi = 2050. * u.year
a = 0.36 # * u.au
e = 0.4
i = -75 * u.deg
w = 171.15 * u.deg
anode = 104.15 * u.deg

epochs = np.linspace(tperi, tperi+P, 100, endpoint='true')
# draw the orbit
_, _, Xorb, Yorb, Zorb, _, _, _ = kep3d(epochs,P,tperi,a,e,i,w,anode)

epochs = np.linspace(tperi, tperi+P, 250, endpoint='true')

# calc the orbit
_, _, Xsa, Ysa, Zsa, _, _, _ = kep3d(epochs,P,tperi,a,e,i,w,anode)

# calculate scattering angle
scang = xyztoscatter(Xsa,Ysa,Zsa)

for i, (xpp,ypp,illum_ang) in enumerate(zip(Xsa, Ysa, scang)):

    # create a figure and axes
    fig, ax1 = plt.subplots(1,1,figsize=(6,6))
    ax1.axis('equal')
    
    ax1.set_xlim(-0.6,0.6)
    ax1.set_ylim(-0.6,0.6)
    # plot the orbit
    ax1.plot(Xorb,Yorb,zorder=-10)

    fname = f'anim{i+1:04d}.jpg'
    print(fname)
    angrot = np.arctan2(xpp,ypp) + np.pi*u.rad
    moon(ax1,xpp.value,ypp.value,rad=0.03,
         scatterang=illum_ang,ang=angrot)

    # Inner working angle
    patch = patches.Circle((0, 0), radius=iwa, color='gray',alpha=0.5)
    ax1.add_patch(patch)
    ax1.scatter(0,0,color='black')

    ax1.set_xlabel('dx [arcsec]',fontsize=16)
    ax1.set_ylabel('dy [arcsec]',fontsize=16)
    plt.draw()
    plt.savefig(fname)
    plt.close()

# calculate current projected separation
rho = np.sqrt(Xsa*Xsa+Ysa*Ysa)

# distance from star to planet
rad_dist = np.sqrt(Xsa*Xsa+Ysa*Ysa+Zsa*Zsa)

visible = (rho > iwa)

The resultant `anim0000.png` files can be buint into an animation using `ffmpeg` and one of the commands below:



Make a lossless jpg animation:

    
    ffmpeg -framerate 30 -i anim%04d.jpg -codec copy planet_phases.mkv

Make an animated gif:

    ffmpeg -i anim%04d.jpg planet_phases.gif
